In [1]:
def calcAndAppendStatValForScan(alleleCounts, snpLocs, statName, subWinStart, subWinEnd, statVals):
    if statName == "tajD":
        statVals[statName].append(allel.stats.diversity.tajima_d(
            alleleCounts, pos=snpLocs, start=subWinStart, stop=subWinEnd))
    elif statName == "pi":
        statVals[statName].append(allel.stats.diversity.sequence_diversity(  # NOQA
            snpLocs, alleleCounts, start=subWinStart, stop=subWinEnd))  # NOQA
    elif statName == "thetaW":
        statVals[statName].append(allel.stats.diversity.watterson_theta(
            snpLocs, alleleCounts, start=subWinStart, stop=subWinEnd))  # NOQA
    elif statName == "thetaW":
        statVals[statName][instanceIndex].append(allel.stats.diversity.watterson_theta(
            snpLocs, alleleCounts, start=subWinStart, stop=subWinEnd))

In [4]:
import numpy as np
import pandas as pd
import allel
from allel.model.ndarray import SortedIndex
from allel.util import asarray_ndim
import re
import subprocess
import os.path

datadir = "/media/kevin/TOSHIBA_EXT/TTT_RecombinationGenomeScans/results_final/"
cmd      = 'ls ' + datadir + '*.vcf.gz'
fileList =  subprocess.run(cmd, shell = True, stdout=subprocess.PIPE).stdout.decode('utf-8')
fileList = fileList.split("\n")
fileList = list(filter(None, fileList))

winsize = 1000     # how big should the sliding window be around each SNP?

In [6]:
for f in fileList:
    print("Processing " + f)
    vcf     = allel.read_vcf(f, fields = ["CHROM", "POS", "GT"])
    m       = re.search('[0-9]{5}', f)
    simNum  = m.group(0)
    
    g       = allel.GenotypeArray(vcf["calldata/GT"])
    ac      = g.count_alleles()
    pos     = vcf['variants/POS']
    
    statvals = { "tajD"   : [],
                 "pi"     : [],
                 "thetaW" : []}


    for SNP in pos:
        winStart = SNP - (winsize/2)
        winEnd = SNP + (winsize/2)
        if winStart < 0:
            winStart = pos[0]
    
        calcAndAppendStatValForScan(ac, pos, 'tajD', winStart, winEnd, statvals)
        calcAndAppendStatValForScan(ac, pos, 'pi', winStart, winEnd, statvals)
        calcAndAppendStatValForScan(ac, pos, 'thetaW', winStart, winEnd, statvals)
        
    scanResultsFile = datadir + simNum + "_Invers_ScanResults.txt"
    outfile         = datadir + simNum + "_Invers_ScanResults_new.txt"
    
    allFeatures = []
    with open (scanResultsFile, 'r') as f:
        for line in f:
            features = line.split()
            features = [f.strip('"') for f in features]
            allFeatures.append(features)
        stats  = allFeatures[1:]
        header = allFeatures[0]
        featDf =  pd.DataFrame(stats, columns = header)
        featDf = featDf[featDf.keep_loci == "TRUE"].reset_index(drop = True)

    featDf = featDf.join(pd.DataFrame.from_dict(statvals))
    featDf.to_csv(outfile, sep = " ", index = False)

Processing /media/kevin/TOSHIBA_EXT/TTT_RecombinationGenomeScans/results_final/10900_Invers_VCFallFILT.vcf.gz
Processing /media/kevin/TOSHIBA_EXT/TTT_RecombinationGenomeScans/results_final/10901_Invers_VCFallFILT.vcf.gz
Processing /media/kevin/TOSHIBA_EXT/TTT_RecombinationGenomeScans/results_final/10902_Invers_VCFallFILT.vcf.gz
Processing /media/kevin/TOSHIBA_EXT/TTT_RecombinationGenomeScans/results_final/10903_Invers_VCFallFILT.vcf.gz
Processing /media/kevin/TOSHIBA_EXT/TTT_RecombinationGenomeScans/results_final/10904_Invers_VCFallFILT.vcf.gz
Processing /media/kevin/TOSHIBA_EXT/TTT_RecombinationGenomeScans/results_final/10905_Invers_VCFallFILT.vcf.gz
Processing /media/kevin/TOSHIBA_EXT/TTT_RecombinationGenomeScans/results_final/10906_Invers_VCFallFILT.vcf.gz
Processing /media/kevin/TOSHIBA_EXT/TTT_RecombinationGenomeScans/results_final/10907_Invers_VCFallFILT.vcf.gz
Processing /media/kevin/TOSHIBA_EXT/TTT_RecombinationGenomeScans/results_final/10908_Invers_VCFallFILT.vcf.gz
Processing